In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [3]:
from langchain.prompts import load_prompt, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

In [4]:
template = '''
I want you to answer the question from the given documentation. 
Even if it's not the correct answer, if it's relevant to the question, answer it. 
If there is nothing relevant in the document, return "No useful information".

Question: {question}
Document: {document}
Answer:
'''

In [5]:
from models.llm.chain import BaseChain
from typing import List

In [6]:
class UnifiedSummaryChunkChain(BaseChain):
    def __init__(self, 
                summary_chunk_template=None, 
                input_variables:List[str]=None,
                summary_chunk_template_path='../openai_skt/models/templates/unified_summary_chunk_prompt.json', 
                model='gpt-3.5-turbo-16k', 
                verbose=False) -> None:
        super().__init__(template=summary_chunk_template, input_variables=input_variables, template_path=summary_chunk_template_path, model=model, verbose=verbose)

    def run(self, chunk:str=None, question:str=None):
        return super().run(document=chunk, question=question)
    
    async def arun(self,chunk:str=None, question:str=None):
        return await super().arun(document=chunk, question=question)

In [7]:
summary_chain = UnifiedSummaryChunkChain(summary_chunk_template=template, input_variables=["question", "document"], verbose=True)

In [8]:
result = summary_chain.run(question="What is the meaning of life?", chunk="The meaning of life is to be happy.") 



> Entering new LLMChain chain...
Prompt after formatting:

I want you to answer the question from the given documentation. 
Even if it's not the correct answer, if it's relevant to the question, answer it. 
If there is nothing relevant in the document, return "nothing relevant".

Question: What is the meaning of life?
Document: The meaning of life is to be happy.
Answer:


> Finished chain.


In [9]:
print(result)

The meaning of life is to be happy.
